<a href="https://colab.research.google.com/github/Youssefelsayed148/Text_Summarizer/blob/main/Good_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
import os
import re
import pandas as pd
import evaluate
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
)
from datasets import load_dataset

# Import Data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/newspaper-text-summarization-cnn-dailymail


In [ ]:
train_df = pd.read_csv(os.path.join(path, "cnn_dailymail", "train.csv"))
validation_df = pd.read_csv(os.path.join(path, "cnn_dailymail", "validation.csv"))
test_df = pd.read_csv(os.path.join(path, "cnn_dailymail", "test.csv"))



# Explore data and Preparing

In [ ]:
train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
test_df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [ ]:
validation_df.head()


,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [ ]:
train_df.isna().sum(), validation_df.isna().sum(), test_df.isna().sum()

(id            0
 article       0
 highlights    0
 dtype: int64,
 id            0
 article       0
 highlights    0
 dtype: int64,
 id            0
 article       0
 highlights    0
 dtype: int64)

In [ ]:
train_df=train_df.drop('id',axis=1)
validation_df=validation_df.drop('id',axis=1)
test_df=test_df.drop('id',axis=1)

In [ ]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text.strip())
    return text


In [ ]:
train_df['article'] = train_df['article'].apply(clean_text)
train_df['highlights'] = train_df['highlights'].apply(clean_text)
validation_df['article'] = validation_df['article'].apply(clean_text)
validation_df['highlights'] = validation_df['highlights'].apply(clean_text)
test_df['article'] = test_df['article'].apply(clean_text)
test_df['highlights'] = test_df['highlights'].apply(clean_text)

In [ ]:
train_df= train_df.rename(columns={'article': 'text', 'highlights': 'summary'})
validation_df= validation_df.rename(columns={'article': 'text', 'highlights': 'summary'})
test_df= test_df.rename(columns={'article': 'text', 'highlights': 'summary'})

In [ ]:
train_df.shape, validation_df.shape, test_df.shape

((287113, 2), (13368, 2), (11490, 2))

In [ ]:
small_train_df = train_df.sample(frac=0.005, random_state=42)
small_validation_df = validation_df.sample(frac=0.040, random_state=42)
small_test_df = test_df.sample(frac=0.040, random_state=42)


In [ ]:
small_train_df.shape, small_validation_df.shape, small_test_df.shape

((1436, 2), (535, 2), (460, 2))

In [ ]:
raw_dataset = DatasetDict({
    'train': Dataset.from_pandas(small_train_df),
    'validation': Dataset.from_pandas(small_validation_df),
    'test': Dataset.from_pandas(small_test_df)
})

# Modelling

In [ ]:
checkpoint= 'facebook/bart-base'
model= AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer= AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def preprocess(example):
    # Tokenize the input text
    model_inputs = tokenizer(
        example['text'],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenize the target summary text
    with tokenizer.as_target_tokenizer() if hasattr(tokenizer, "as_target_tokenizer") else contextlib.nullcontext():
        labels = tokenizer(
            example['summary'],
            max_length=128,
            truncation=True,
            padding="max_length"
        )["input_ids"]

    # Replace padding token ids in labels with -100
    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]

    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
tokenized_dataset= raw_dataset.map(preprocess,batched=True)

Map:   0%|          | 0/1436 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/535 [00:00<?, ? examples/s]

Map:   0%|          | 0/460 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Make sure predictions are in valid range
    predictions = np.clip(predictions, 0, tokenizer.vocab_size - 1)

    # Replace -100 in labels with pad_token_id
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    # Decode both
    try:
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    except OverflowError as e:
        print("OverflowError during decoding! Debug here:")
        print("Sample predictions:", predictions[0][:20])
        print("Sample labels:", labels[0][:20])
        raise e

    # Compute ROUGE
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v * 100, 2) for k, v in result.items()}


In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
datacollator= DataCollatorForSeq2Seq(tokenizer, model=model)

# Model1

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-news-summarizer",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=2,
    weight_decay=0.01,
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,
    load_best_model_at_end=True,
    save_total_limit=2,
    fp16=True,
)

In [ ]:
trainer= Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=datacollator,
    compute_metrics=None,
)

/tmp/ipython-input-26-3231785285.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer= Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: youssefelsayed1480 (youssefelsayed1480-freelance) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.748784
2,No log,1.301555


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=288, training_loss=2.942008124457465, metrics={'train_runtime': 194.7264, 'train_samples_per_second': 14.749, 'train_steps_per_second': 1.479, 'total_flos': 875581554032640.0, 'train_loss': 2.942008124457465, 'epoch': 2.0})

In [ ]:
test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print("Test ROUGE scores:", test_results)


Test ROUGE scores: {'eval_loss': 1.2920677661895752, 'eval_runtime': 3.4474, 'eval_samples_per_second': 133.433, 'eval_steps_per_second': 13.343, 'epoch': 2.0}


# Model 2

In [ ]:
trainer2= Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=datacollator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-29-495064108.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer2= Seq2SeqTrainer(


In [ ]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,1.113135,40.250000,18.580000,27.580000,27.570000
2,No log,1.099245,40.160000,18.490000,27.660000,27.680000


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=288, training_loss=1.1689149008856878, metrics={'train_runtime': 620.6193, 'train_samples_per_second': 4.628, 'train_steps_per_second': 0.464, 'total_flos': 875581554032640.0, 'train_loss': 1.1689149008856878, 'epoch': 2.0})

In [ ]:
test_results = trainer2.evaluate(eval_dataset=tokenized_dataset["test"])
print("Test ROUGE scores:", test_results)


Test ROUGE scores: {'eval_loss': 1.0816066265106201, 'eval_rouge1': 39.04, 'eval_rouge2': 16.93, 'eval_rougeL': 26.06, 'eval_rougeLsum': 26.11, 'eval_runtime': 183.1196, 'eval_samples_per_second': 2.512, 'eval_steps_per_second': 0.251, 'epoch': 2.0}


In [ ]:
def summarize(text):
    inputs = tokenizer([text], return_tensors="pt", max_length=512, truncation=True)
    # Move input tensors to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    summary_ids = model.generate(
    **inputs,
    max_length=128,
    min_length=40,
    num_beams=4,
    length_penalty=1.5,  # Encourage longer summaries
    early_stopping=True
)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print(test_df['text'].iloc[2])

Dougie Freedman is on the verge of agreeing a new two-year deal to remain at Nottingham Forest. Freedman has stabilised Forest since he replaced cult hero Stuart Pearce and the club's owners are pleased with the job he has done at the City Ground. Dougie Freedman is set to sign a new deal at Nottingham Forest . Freedman has impressed at the City Ground since replacing Stuart Pearce in February . They made an audacious attempt on the play-off places when Freedman replaced Pearce but have tailed off in recent weeks. That has not prevented Forest's ownership making moves to secure Freedman on a contract for the next two seasons.


In [ ]:
print(summarize(test_df['text'].iloc[2]))

Dougie Freedman is on the verge of agreeing a new two-year deal to remain at Nottingham Forest . Freedman has stabilised Forest since he replaced cult hero Stuart Pearce . The club's owners are pleased with the job he has done at the City Ground .


In [ ]:
trainer2.save_model("summarization-model")
tokenizer.save_pretrained("summarization-model")


('summarization-model/tokenizer_config.json',
 'summarization-model/special_tokens_map.json',
 'summarization-model/vocab.json',
 'summarization-model/merges.txt',
 'summarization-model/added_tokens.json',
 'summarization-model/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login,HfApi, create_repo
notebook_login()

api = HfApi()

# Create the repository if it doesn't exist
repo_name = "My-Good-Summarizer"
repo_id = f"Youssef-El-SaYed/{repo_name}" # Replace Youssef-El-SaYed with your Hugging Face username
create_repo(repo_id, repo_type="model", exist_ok=True)

api.upload_folder(
    folder_path="summarization-model",
    repo_id=repo_id,
    repo_type="model"
)

Uploading...:   0%|          | 0.00/558M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Youssef-El-SaYed/My-Good-Summarizer/commit/0d9542036883cae94f34ba38689d93ef24ab808c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0d9542036883cae94f34ba38689d93ef24ab808c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Youssef-El-SaYed/My-Good-Summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='Youssef-El-SaYed/My-Good-Summarizer'), pr_revision=None, pr_num=None)